In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time
import matplotlib.pyplot as plt

In [32]:
symbols = ["H","H","H","H"]
geometry = np.array([[0.0,0.0,-1.4],[0.0, 0.0, -0.70],
                    [0.0, 0.0, 0.70],[0.0,0.0,1.4]])

H, qubits = qml.qchem.molecular_hamiltonian(symbols,geometry,charge=0)

In [33]:
print(H)

  (-1.235732808313587) [Z7]
+ (-1.2357328083135868) [Z6]
+ (-0.47085926142206413) [Z4]
+ (-0.47085926142206413) [Z5]
+ (0.11695724217350083) [Z3]
+ (0.11695724217350087) [Z2]
+ (0.30230459003340443) [Z1]
+ (0.30230459003340454) [Z0]
+ (3.3567991608676246) [I0]
+ (-1.1466383398328617e-12) [Y5 Y7]
+ (-1.1466383398328617e-12) [X5 X7]
+ (0.1016826080575708) [Z0 Z2]
+ (0.1016826080575708) [Z1 Z3]
+ (0.12144222500512473) [Z2 Z4]
+ (0.12144222500512473) [Z3 Z5]
+ (0.12489206480045467) [Z0 Z4]
+ (0.12489206480045467) [Z1 Z5]
+ (0.1258319034074734) [Z4 Z6]
+ (0.1258319034074734) [Z5 Z7]
+ (0.13604538113091058) [Z2 Z6]
+ (0.13604538113091058) [Z3 Z7]
+ (0.14617318053408263) [Z0 Z3]
+ (0.14617318053408263) [Z1 Z2]
+ (0.14947935534397927) [Z0 Z6]
+ (0.14947935534397927) [Z1 Z7]
+ (0.15005473802225783) [Z2 Z5]
+ (0.15005473802225783) [Z3 Z4]
+ (0.15135665764031578) [Z2 Z3]
+ (0.15155637432671126) [Z0 Z5]
+ (0.15155637432671126) [Z1 Z4]
+ (0.15470735190521281) [Z0 Z1]
+ (0.1605624367211701) [Z4 Z5]


In [34]:
print(qubits)

8


In [35]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators,qubits)

In [36]:
for idx, generator in enumerate(generators):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops[idx]}")

generator 1:   (1.0) [Z0 Z2 Z4 Z6], paulix_op: PauliX(wires=[6])
generator 2:   (1.0) [Z1 Z3 Z5 Z7], paulix_op: PauliX(wires=[7])


In [37]:
n_electrons = 4
paulix_sector = qml.qchem.optimal_sector(H,generators,n_electrons)
print(paulix_sector)

[1, 1]


In [38]:
H_tapered = qml.taper(H, generators, paulixops, paulix_sector)
print(H_tapered)

  ((-0.4708592614220639+0j)) [Z4]
+ ((-0.4708592614220639+0j)) [Z5]
+ ((0.11695724217350077+0j)) [Z3]
+ ((0.11695724217350081+0j)) [Z2]
+ ((0.3023045900334043+0j)) [Z1]
+ ((0.30230459003340443+0j)) [Z0]
+ ((3.356799160867623+0j)) [I0]
+ ((-0.04714855568932794+0j)) [Y4 Y5]
+ ((-0.02690255805746411+0j)) [Y2 Y3]
+ ((-0.0202951610848933+0j)) [Y0 Y1]
+ ((-0.011364143471853699+0j)) [Y0 Y5]
+ ((0.14617318053408257+0j)) [Z0 Z3]
+ ((0.14617318053408257+0j)) [Z1 Z2]
+ ((0.15005473802225777+0j)) [Z2 Z5]
+ ((0.15005473802225777+0j)) [Z3 Z4]
+ ((0.15135665764031572+0j)) [Z2 Z3]
+ ((0.1515563743267112+0j)) [Z0 Z5]
+ ((0.1515563743267112+0j)) [Z1 Z4]
+ ((0.15470735190521276+0j)) [Z0 Z1]
+ ((0.16056243672117004+0j)) [Z4 Z5]
+ ((0.2275145114650441+0j)) [Z0 Z2]
+ ((0.2275145114650441+0j)) [Z1 Z3]
+ ((0.26093744593136514+0j)) [Z0 Z4]
+ ((0.26093744593136514+0j)) [Z1 Z5]
+ ((0.2709215803491039+0j)) [Z2 Z4]
+ ((0.2709215803491039+0j)) [Z3 Z5]
+ ((-1.2357328083135866+0j)) [Z1 Z3 Z5]
+ ((-1.2357328083135863+

In [39]:
H_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H), wires=H.wires)
H_tapered_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H_tapered), wires=H_tapered.wires)

print("Eigenvalues of H:\n", qml.eigvals(H_sparse, k=16))
print("\nEigenvalues of H_tapered:\n", qml.eigvals(H_tapered_sparse, k=4))

Eigenvalues of H:
 [-1.47924833 -0.90625234 -0.90625234 -0.15277341 -0.50270799 -0.50270799
 -0.36077427 -0.36077427  0.2082724   0.2396423  -0.39067699 -0.39067699
 -0.39067699  0.01044941  0.01044941  0.01044941]

Eigenvalues of H_tapered:
 [-1.47924833 -0.39067699 -0.15277341  0.01044941]


In [41]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=len(H.wires))
print(state_tapered)

[1 1 1 1 0 0]


In [42]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
tapered_doubles = [ qchem.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles]
tapered_singles = [
    qchem.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles]

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)


In [43]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

for n in range(1, 41):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if not n % 5:
        print(f"n: {n}, E: {energy:.8f} Ha, Params: {params}")

n: 5, E: -1.47921887 Ha, Params: [ 6.17315257e-02 -2.09935815e-18 -3.78978407e-18  2.87686705e-02
 -2.29501586e-02  6.29780881e-18  4.63649364e-02 -2.34162979e-02
 -7.81597272e-18  7.96496141e-18 -2.34163076e-02  4.63646874e-02
 -6.50982882e-18 -2.29494819e-02  9.38143591e-02  9.43250629e-18
  7.56971477e-18  3.71834791e-02  2.09395739e-03  1.42266322e-16
 -2.09399548e-03  2.37497484e-16 -1.23582970e-17 -3.93236019e-06
 -1.59909718e-17  3.95553302e-06]
n: 10, E: -1.47924408 Ha, Params: [ 6.13678559e-02 -2.35692236e-18 -6.49806157e-18  2.67199900e-02
 -2.29904170e-02  7.52172298e-18  4.66819697e-02 -2.36929661e-02
 -9.22454155e-18  1.86381227e-17 -2.36929654e-02  4.66819844e-02
 -1.16902320e-17 -2.29904571e-02  9.47158483e-02  1.82486094e-17
  7.73578672e-18  3.62344112e-02  2.78143770e-03  3.60117268e-16
 -2.78143879e-03  4.75613348e-16 -3.11187983e-17 -1.21277627e-04
 -4.04881742e-17  1.21276201e-04]
n: 15, E: -1.47924422 Ha, Params: [ 6.13750219e-02 -2.07654071e-18  1.09397589e-17  2

In [44]:
print(qml.draw(tapered_circuit)(params))

0: ─╭BasisState(M0)─╭Exp(0.00+0.00j X@X@X@Y)─╭Exp(0.00+0.00j X@X@Y@X)─╭Exp(-0.00-0.00j X@Y@X@X)
1: ─├BasisState(M0)─├Exp(0.00+0.00j X@X@X@Y)─├Exp(0.00+0.00j X@X@Y@X)─├Exp(-0.00-0.00j X@Y@X@X)
2: ─├BasisState(M0)─│────────────────────────│────────────────────────│────────────────────────
3: ─├BasisState(M0)─│────────────────────────│────────────────────────│────────────────────────
4: ─├BasisState(M0)─├Exp(0.00+0.00j X@X@X@Y)─├Exp(0.00+0.00j X@X@Y@X)─├Exp(-0.00-0.00j X@Y@X@X)
5: ─╰BasisState(M0)─╰Exp(0.00+0.00j X@X@X@Y)─╰Exp(0.00+0.00j X@X@Y@X)─╰Exp(-0.00-0.00j X@Y@X@X)

──╭Exp(0.00+0.00j X@Y@Y@Y)─╭Exp(-0.00-0.00j Y@X@X@X)─╭Exp(0.00+0.00j Y@X@Y@Y)
──├Exp(0.00+0.00j X@Y@Y@Y)─├Exp(-0.00-0.00j Y@X@X@X)─├Exp(0.00+0.00j Y@X@Y@Y)
──│────────────────────────│─────────────────────────│───────────────────────
──│────────────────────────│─────────────────────────│───────────────────────
──├Exp(0.00+0.00j X@Y@Y@Y)─├Exp(-0.00-0.00j Y@X@X@X)─├Exp(0.00+0.00j Y@X@Y@Y)
──╰Exp(0.00+0.00j X@Y@Y@Y)─╰Exp(-